In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as pl

In [ ]:
df = pd.read_csv("/kaggle/input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
sns.distplot(df['CO2 Emissions(g/km)'])

In [ ]:
corr = df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr,annot=True)

In [ ]:
plt.figure(figsize=(25,5))
df.groupby(['Fuel Consumption City (L/100 km)'])['CO2 Emissions(g/km)'].mean().sort_values().plot(kind='bar')
plt.xticks(rotation=90, horizontalalignment='center', fontweight='light', fontsize='7')
plt.ylabel("CO2 Emissions(g/km)")

In [ ]:
plt.figure(figsize=(20,5))
df.groupby(['Fuel Consumption Comb (mpg)'])['CO2 Emissions(g/km)'].mean().sort_values().plot(kind='bar')
plt.xticks(rotation=90, horizontalalignment='center', fontweight='light', fontsize='7')
plt.ylabel("CO2 Emissions(g/km)")

In [ ]:
Ft = pd.get_dummies(df['Fuel Type'],drop_first=True,prefix='Fuel')
df = df.drop(['Fuel Type'],axis=1)
df = pd.concat([df,Ft],axis=1)
Tr = pd.get_dummies(df['Transmission'],drop_first=True)
df = df.drop(['Transmission'],axis=1)
df = pd.concat([df,Tr],axis=1)

In [ ]:
df.head()

In [ ]:
X = df.drop(['CO2 Emissions(g/km)','Fuel Consumption Comb (mpg)'],axis=1)
y = df['CO2 Emissions(g/km)']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.5,random_state=42)

In [ ]:
cat_cols = ['Make','Model','Vehicle Class']
import category_encoders as ce
target_enc = ce.CatBoostEncoder(cols = cat_cols)
target_enc.fit(x_train[cat_cols],y_train)

In [ ]:
train_enc = target_enc.transform(x_train[cat_cols])
test_enc = target_enc.transform(x_test[cat_cols])

x_train = x_train.drop(['Make','Model','Vehicle Class'],axis=1)
x_test = x_test.drop(['Make','Model','Vehicle Class'],axis=1)
x_train = pd.concat([x_train,train_enc],axis=1)
x_test = pd.concat([x_test,test_enc],axis=1)


In [ ]:
x_train.head()

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model = lr.fit(x_train,y_train)
y_pred = model.predict(x_test) 
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
mae = mean_absolute_error(y_test,y_pred)
print('Absolute error:',mae)
mre = mean_squared_error(y_test,y_pred)
print('Squared error',mre)

In [ ]:
#Linear
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(121)
plt.scatter(y_test, y_pred, alpha=0.1, color = 'g')
ax2 = fig.add_subplot(122)
sns.distplot(y_test-y_pred)

In [ ]:
#Ridge regression with multiple features by using the SCIKIT-LEARN library
from sklearn.linear_model import Ridge
ridge = Ridge()
model2 = ridge.fit(x_train,y_train)
y_pred2 = model2.predict(x_test)
r2_score(y_test,y_pred2)

In [ ]:
y_test


In [ ]:
y_pred2

In [ ]:
mse = mean_squared_error(y_test,y_pred2)
mse

In [ ]:
mae = mean_absolute_error(y_test,y_pred2)
mae

In [ ]:
#Ridge
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(121)
plt.scatter(y_test, y_pred2, alpha=0.1, color = 'g')
ax2 = fig.add_subplot(122)
sns.distplot(y_test-y_pred2)

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso()
model3=lasso.fit(x_train, y_train)
y_pred3 = model3.predict(x_test)
r2_score(y_test,y_pred3)

In [ ]:
y_pred3

In [ ]:
y_test

In [ ]:
mse = mean_squared_error(y_test,y_pred3)
mse

In [ ]:
mae = mean_absolute_error(y_test,y_pred3)
mae

In [ ]:
#Lasso
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(121)
plt.scatter(y_test, y_pred3, alpha=0.1, color = 'g')
ax2 = fig.add_subplot(122)
sns.distplot(y_test-y_pred3)

In [ ]:
from sklearn.linear_model import ElasticNet
elasticNet= ElasticNet()
model4=elasticNet.fit(x_train, y_train)
y_pred4 = model4.predict(x_test)
r2_score(y_test,y_pred4)

In [ ]:
y_pred4

In [ ]:
y_test

In [ ]:
mse = mean_squared_error(y_test,y_pred4)
mse

In [ ]:
mae = mean_absolute_error(y_test,y_pred4)
mae

In [ ]:
#ElasticNet
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(121)
plt.scatter(y_test, y_pred4, alpha=0.1, color = 'g')
ax2 = fig.add_subplot(122)
sns.distplot(y_test-y_pred4)

In [ ]:
from sklearn.ensemble import VotingRegressor
votingReg=VotingRegressor([('LinearRegression',lr),('lasso',lasso),('Ridge',ridge),('ElasticNet',elasticNet)]).fit(x_train, y_train)
y_pred = votingReg.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mse


In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mae

In [ ]:
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(121)
plt.scatter(y_test, y_pred, alpha=0.1, color = 'g')
ax2 = fig.add_subplot(122)
sns.distplot(y_test-y_pred)